João Inácio Scrimini - 202370701

Usei o VS Code, com diretório direto do ambiente em que estava o arquivo.

Criando as tabelas SQL:

In [2]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('tb4_2.db')

In [2]:
comando_cria_tabela_indicador= "\
CREATE TABLE IF NOT EXISTS indicador (\
id_indicador INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,\
cod_indicador text NOT NULL,\
nome_indicador text NOT NULL,\
UNIQUE(cod_indicador)\
);"
comando_cria_tabela_pais= "\
CREATE TABLE IF NOT EXISTS pais (\
id_pais INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,\
cod_pais text NOT NULL,\
nomec_pais text NOT NULL,\
nomel_pais text NOT NULL,\
regiao_pais text NOT NULL,\
UNIQUE(cod_pais)\
);"
comando_cria_tabela_pais_indicador= "\
CREATE TABLE IF NOT EXISTS pais_indicador (\
id_pais_indicador INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,\
ano INTEGER NOT NULL,\
valor REAL NOT NULL,\
id_pais INTEGER NOT NULL,\
id_indicador INTEGER NOT NULL,\
UNIQUE(id_pais,id_indicador,ano),\
FOREIGN KEY (id_pais) REFERENCES pais ON DELETE CASCADE ON UPDATE CASCADE,\
FOREIGN KEY (id_indicador) REFERENCES indicador ON DELETE CASCADE ON UPDATE CASCADE\
);"

conn.executescript( comando_cria_tabela_indicador )
conn.executescript( comando_cria_tabela_pais )
conn.executescript( comando_cria_tabela_pais_indicador )

In [3]:
registros_indicador = pd.read_sql_query("SELECT * FROM indicador", conn )
display(registros_indicador)

registros_pais = pd.read_sql_query("SELECT * FROM pais", conn )
display(registros_pais)

registros_pais_indicador = pd.read_sql_query("SELECT * FROM pais_indicador", conn )
display(registros_pais_indicador)

,id_indicador,cod_indicador,nome_indicador


,id_pais,cod_pais,nomec_pais,nomel_pais,regiao_pais


,id_pais_indicador,ano,valor,id_pais,id_indicador


In [4]:
tabelas = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", conn )
display(tabelas)

,name
0,indicador
1,sqlite_sequence
2,pais
3,pais_indicador


In [5]:
conn.commit()
conn.close()

Importando os dados:

tabela indicador -

In [6]:
ds_ind=pd.read_csv("dataset_bancomundial.csv", sep=';')

ds_ind_red=ds_ind[ ['Indicator Code', 'Indicator Name'] ].copy()
ds_ind_red.sort_values(by='Indicator Code', inplace=True)

ds_ind_sem_rep=pd.DataFrame()
ds_ind_sem_rep['cod_indicador']=ds_ind_red['Indicator Code'].unique()
ds_ind_sem_rep['nome_indicador']=ds_ind_red['Indicator Name'].unique()
display(ds_ind_sem_rep)

,cod_indicador,nome_indicador
0,GFDD.AI.01,"Bank accounts per 1,000 adults"
1,GFDD.AI.02,"Bank branches per 100,000 adults"
2,GFDD.AI.03,Firms with a bank loan or line of credit (%)
3,GFDD.AI.04,Small firms with a bank loan or line of credit...
4,GFDD.AI.05,Account at a formal financial institution (% a...
...,...,...
110,GFDD.SM.01,Stock price volatility
111,NY.GDP.MKTP.CD,GDP (Current USD)
112,NY.GDP.PCAP.KD,GDP per capita (Constant 2005 USD)
113,NY.GNP.MKTP.CD,GNI (Current USD)


In [7]:
comando_insere_dados_tabela_indicador=''

for coluna,linha in ds_ind_sem_rep.iterrows():
  insere_linha=f"INSERT INTO indicador (cod_indicador, nome_indicador) VALUES (\"{linha['cod_indicador']}\",\"{linha['nome_indicador']}\");"
  comando_insere_dados_tabela_indicador= comando_insere_dados_tabela_indicador + '\n' + insere_linha



In [8]:
conn = sqlite3.connect('tb4_2.db')
conn.executescript("UPDATE SQLITE_SEQUENCE SET SEQ=0 WHERE NAME='indicador';")
conn.executescript(comando_insere_dados_tabela_indicador)

In [9]:
registros_indicador = pd.read_sql_query("SELECT * FROM indicador", conn )
display(registros_indicador)

,id_indicador,cod_indicador,nome_indicador
0,1,GFDD.AI.01,"Bank accounts per 1,000 adults"
1,2,GFDD.AI.02,"Bank branches per 100,000 adults"
2,3,GFDD.AI.03,Firms with a bank loan or line of credit (%)
3,4,GFDD.AI.04,Small firms with a bank loan or line of credit...
4,5,GFDD.AI.05,Account at a formal financial institution (% a...
...,...,...,...
110,111,GFDD.SM.01,Stock price volatility
111,112,NY.GDP.MKTP.CD,GDP (Current USD)
112,113,NY.GDP.PCAP.KD,GDP per capita (Constant 2005 USD)
113,114,NY.GNP.MKTP.CD,GNI (Current USD)


In [10]:
conn.commit()
conn.close()

tabela pais -

In [11]:
ds_pais=pd.read_csv("dataset_bancomundial_paises.csv", sep=';')
ds_pais=ds_pais.sort_values(['Country Code'], ascending=[ True]).drop(['index'],1)
ds_pais

C:\Users\joaoi\AppData\Local\Temp\ipykernel_13320\657619491.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  ds_pais=ds_pais.sort_values(['Country Code'], ascending=[ True]).drop(['index'],1)


,Country Code,Short Name,Long Name,Region
8,ABW,Aruba,Aruba,Latin America & Caribbean
3,ADO,Andorra,Principality of Andorra,Europe & Central Asia
0,AFG,Afghanistan,Islamic State of Afghanistan,South Asia
4,AGO,Angola,People's Republic of Angola,Sub-Saharan Africa
1,ALB,Albania,Republic of Albania,Europe & Central Asia
...,...,...,...,...
214,YEM,Yemen,Republic of Yemen,Middle East & North Africa
174,ZAF,South Africa,Republic of South Africa,Sub-Saharan Africa
49,ZAR,Dem. Rep. Congo,Democratic Republic of the Congo,Sub-Saharan Africa
215,ZMB,Zambia,Republic of Zambia,Sub-Saharan Africa


In [12]:
comando_insere_dados_tabela_pais=''

for coluna,linha in ds_pais.iterrows():
  #print(cod, nomec, nomel, regiao)
  insere_linha=f"INSERT INTO pais (cod_pais, nomec_pais, nomel_pais, regiao_pais) VALUES (\"{linha['Country Code']}\",\
    \"{linha['Short Name']}\",\"{linha['Long Name']}\",\"{linha['Region']}\");"
  comando_insere_dados_tabela_pais= comando_insere_dados_tabela_pais + '\n' + insere_linha

In [13]:
conn = sqlite3.connect('tb4_2.db')
conn.executescript("UPDATE SQLITE_SEQUENCE SET SEQ=0 WHERE NAME='pais';")
conn.executescript(comando_insere_dados_tabela_pais)

In [14]:
registros_pais = pd.read_sql_query("SELECT * FROM pais", conn )
display(registros_pais)

,id_pais,cod_pais,nomec_pais,nomel_pais,regiao_pais
0,1,ABW,Aruba,Aruba,Latin America & Caribbean
1,2,ADO,Andorra,Principality of Andorra,Europe & Central Asia
2,3,AFG,Afghanistan,Islamic State of Afghanistan,South Asia
3,4,AGO,Angola,People's Republic of Angola,Sub-Saharan Africa
4,5,ALB,Albania,Republic of Albania,Europe & Central Asia
...,...,...,...,...,...
212,213,YEM,Yemen,Republic of Yemen,Middle East & North Africa
213,214,ZAF,South Africa,Republic of South Africa,Sub-Saharan Africa
214,215,ZAR,Dem. Rep. Congo,Democratic Republic of the Congo,Sub-Saharan Africa
215,216,ZMB,Zambia,Republic of Zambia,Sub-Saharan Africa


In [15]:
conn.commit()
conn.close()

tabela indicador e pais -

In [16]:
conn = sqlite3.connect('tb4_2.db')

registros_pais2 = pd.read_sql_query("SELECT id_pais, cod_pais FROM pais", conn )
display(registros_pais2)

registros_indicador2=pd.read_sql_query("SELECT id_indicador, cod_indicador FROM indicador", conn )
display(registros_indicador2)

,id_pais,cod_pais
0,1,ABW
1,2,ADO
2,3,AFG
3,4,AGO
4,5,ALB
...,...,...
212,213,YEM
213,214,ZAF
214,215,ZAR
215,216,ZMB


,id_indicador,cod_indicador
0,1,GFDD.AI.01
1,2,GFDD.AI.02
2,3,GFDD.AI.03
3,4,GFDD.AI.04
4,5,GFDD.AI.05
...,...,...
110,111,GFDD.SM.01
111,112,NY.GDP.MKTP.CD
112,113,NY.GDP.PCAP.KD
113,114,NY.GNP.MKTP.CD


In [17]:
ds_ind2=pd.read_csv("dataset_bancomundial.csv", sep=';', decimal=',')
ds_ind_red2=ds_ind2.drop( columns=['index', 'Country Name', 'Indicator Name'] )
ds_ind_red2=ds_ind_red2.sort_values( ['Country Code', 'Indicator Code'], ascending=[ True, True ] )

In [18]:
comando_insere_dados_tabela_pais_indicador=''

for i, linha in registros_indicador2.iterrows():
  paises_sel=ds_ind_red2[ ds_ind_red2['Indicator Code']==linha['cod_indicador'] ].copy()
  paises_sel.sort_values(by=['Country Code'], inplace=True)

  # insere as colunas adicionais
  paises_sel['cod']=registros_pais2['cod_pais'].to_numpy()
  paises_sel['comp_cod']=(paises_sel['Country Code']==paises_sel['cod'])

  if( len( paises_sel[ ~paises_sel['comp_cod'] ] )<=0 ):
    # remove as colunas que foram utilizadas para comparacao
    paises_sel.drop( columns=['comp_cod','cod'], inplace=True )
    paises_sel.insert( loc=2, column='id_pais', value=registros_pais2['id_pais'].to_numpy() )
    paises_sel.insert( loc=3, column='id_indicador', value=linha['id_indicador'] )
    paises_sel_anos=paises_sel.melt( id_vars=['Country Code', 'Indicator Code', 'id_pais', 'id_indicador'], var_name='ano_indicador', value_name='valor_indicador'  )
    paises_sel_anos_validos=paises_sel_anos[ ~paises_sel_anos['valor_indicador'].isna() ].copy()
    paises_sel_anos_validos['comando']='COMANDO'
    paises_sel_anos_validos['fim']='FIM'
    paises_sel_anos_validos=paises_sel_anos_validos[['comando','ano_indicador','valor_indicador','id_pais','id_indicador','fim']]
    paises_sel_anos_validos.to_csv('comando_sql_pais_indicador.txt', sep=',', header=False, index=False)
    comando_sql_pais_indicador_arq=open('comando_sql_pais_indicador.txt','r')
    comando_sql_pais_indicador=comando_sql_pais_indicador_arq.read()
    comando_sql_pais_indicador_arq.close()

    # substitui as strings nos dados a serem incluidos na tabela "dados"
    comando_sql_pais_indicador=comando_sql_pais_indicador.replace('\n',',')
    comando_sql_pais_indicador=comando_sql_pais_indicador+"FINAL"
    comando_sql_pais_indicador=comando_sql_pais_indicador.replace(',','\",\"')
    comando_sql_pais_indicador=comando_sql_pais_indicador.replace(',\"FIM','),\n')
    comando_sql_pais_indicador=comando_sql_pais_indicador.replace('\",\"COMANDO\",','(')
    comando_sql_pais_indicador=comando_sql_pais_indicador.replace('COMANDO\",','(')
    comando_sql_pais_indicador=comando_sql_pais_indicador.replace(',\n\",\"FINAL',';')
    comando_sql_pais_indicador='INSERT INTO pais_indicador (ano, valor, id_pais, id_indicador) VALUES \n'+comando_sql_pais_indicador
    #print(comando_sql_pais_indicador)
    conn.executescript(comando_sql_pais_indicador)
  else:
    print('O indicador ',linha['cod_indicador'], ' nao tem todos os paises')

In [19]:
registros_pais_indicador = pd.read_sql_query("SELECT * FROM pais_indicador", conn )
display(registros_pais_indicador)

,id_pais_indicador,ano,valor,id_pais,id_indicador
0,1,2001,7.157217e-01,31,1
1,2,2001,0.000000e+00,40,1
2,3,2001,2.651506e+02,54,1
3,4,2001,9.959111e+00,68,1
4,5,2001,3.578609e-01,113,1
...,...,...,...,...,...
294736,294737,2013,2.440738e+07,213,115
294737,294738,2013,5.315749e+07,214,115
294738,294739,2013,6.751368e+07,215,115
294739,294740,2013,1.453864e+07,216,115


In [20]:
conn.executescript("UPDATE SQLITE_SEQUENCE SET SEQ=0 WHERE NAME='pais_indicador';")

In [21]:
conn.commit()
conn.close()

salvar as tabelas:

In [22]:
conn = sqlite3.connect('tb4_2.db')

query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(query, conn)
table_names = tables['name'].tolist()

with pd.ExcelWriter('tb4_2.xlsx') as writer:
    for table_name in table_names:
        df = pd.read_sql_query(f"SELECT * FROM {table_name} LIMIT 1000", conn)
        df.to_excel(writer, sheet_name=table_name, index=False)

# Fechar a conexão com o banco de dados
conn.close()

OBS: O moodle não permite enviar arquivos maiores que 5MB, então o arquivo excel não pode ser enviado. Então salvarei apenas com as 1000 primeiras obs:

In [3]:
conn = sqlite3.connect('tb4_2.db')

query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(query, conn)
table_names = tables['name'].tolist()

with pd.ExcelWriter('tb4_2_1000.xlsx') as writer:
    for table_name in table_names:
        df = pd.read_sql_query(f"SELECT * FROM {table_name} LIMIT 1000", conn)
        df.to_excel(writer, sheet_name=table_name, index=False)

# Fechar a conexão com o banco de dados
conn.close()